<a href="https://colab.research.google.com/github/KimManSub/opencv_practice/blob/main/%EA%B0%9D%EC%B2%B4_%EC%B6%94%EC%A0%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Drive 마운트
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks


# YOLO 모델 파일 다운로드 (이미 필요한 파일을 다운로드)
!wget https://pjreddie.com/media/files/yolov3.weights -O /content/yolov3.weights
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg -O /content/yolov3.cfg
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names -O /content/coco.names


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks
--2024-12-30 18:27:40--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘/content/yolov3.weights’

/content/yolov3.wei 100%[===================>] 236.52M  34.7MB/s    in 7.2s    

2024-12-30 18:27:47 (32.7 MB/s) - ‘/content/yolov3.weights’ saved [248007048/248007048]

--2024-12-30 18:27:47--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
H

In [ ]:
import torch
print(torch.cuda.is_available())  # True이면 GPU가 활성화된 상태

True


In [ ]:
import cv2
import numpy as np
from google.colab import drive
from google.colab.patches import cv2_imshow

# YOLO 모델 로드
yolo_net = cv2.dnn.readNet('/content/yolov3.weights', '/content/yolov3.cfg')

# GPU 사용을 위한 설정 (OpenCV DNN 모듈에서 GPU 사용)
yolo_net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)  # CUDA 백엔드 활성화
yolo_net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)    # GPU에서 실행하도록 설정

layer_names = yolo_net.getLayerNames()
output_layers = yolo_net.getUnconnectedOutLayersNames()

# COCO 클래스 라벨 로드 (차량 감지)
with open('/content/coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# 차량 감지 함수
def detect_vehicles(frame):
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    yolo_net.setInput(blob)
    outs = yolo_net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id == 2:  # 클래스 ID 2는 차량 클래스 (Car)
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    return boxes, confidences, class_ids

def main():
    # 비디오 파일 경로 설정
    video_path = "/content/drive/MyDrive/Colab Notebooks/이미지처리/car.mp4"
    output_path = "/content/drive/MyDrive/Colab Notebooks/car_tracking_output.mp4"

    # 비디오 캡처 객체 생성
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: 비디오 파일을 열 수 없습니다.")
        return

    # 비디오 속성 가져오기
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = 15  # 15fps로 처리하도록 수정

    # 출력 비디오 설정 (해상도를 640x480으로 줄임)
    resized_width = 640
    resized_height = 480
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(output_path, fourcc, fps, (resized_width, resized_height))

    # 추적기 목록
    trackers = []
    detection_interval = 15  # 감지 주기를 설정 (15프레임마다 재감지)

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 프레임 수 줄이기 (15fps로 처리)
        frame_count += 1
        if frame_count % (30 // 15) != 0:  # 15fps로 처리 (원래 FPS가 30이라면, 30fps에서 15fps로 변경)
            continue

        # 프레임 해상도 줄이기 (640x480으로 리사이즈)
        frame = cv2.resize(frame, (resized_width, resized_height))

        # 일정 주기마다 재감지 (예: 15프레임마다)
        if frame_count % detection_interval == 0 or len(trackers) == 0:
            boxes, confidences, class_ids = detect_vehicles(frame)

            # 비최대 억제 (NMS)
            indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.4)

            # 추적기 초기화
            if len(indices) > 0:
                for i in indices.flatten():
                    x, y, w, h = boxes[i]
                    # 각 차량에 대해 추적기 생성 및 초기화
                    tracker = cv2.TrackerCSRT_create()  # 다른 추적기도 가능 (KCF, MOSSE 등)
                    tracker.init(frame, (x, y, w, h))
                    trackers.append(tracker)

        else:
            # 추적기 업데이트
            for i, tracker in enumerate(trackers):
                success, bbox = tracker.update(frame)

                if success:
                    x, y, w, h = [int(v) for v in bbox]
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, "Car", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                else:
                    cv2.putText(frame, "Lost", (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        # 프레임 저장
        out.write(frame)

    # 리소스 해제
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print("차량 추적 완료")

    # 비디오 파일 다운로드 링크 생성
    from google.colab import files
    files.download(output_path)  # 결과 비디오 파일 다운로드

if __name__ == "__main__":
    main()
